In [1]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

In [2]:
# Adapted from http://www.davidsbatista.net/blog/2018/02/23/model_optimization/

class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [3]:
models1 = {
    "Ridge": Ridge(),
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "RandomForestRegressor": RandomForestRegressor(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
}

params1 = {
    "Ridge": {
        "alpha": [1, 0.1, 0.01, 0.001, 0.0001, 0],
        "fit_intercept": [True, False],
        "solver": ["svd", "cholesky", "sparse_cg", "sag"],
    },
    "LinearRegression": {"fit_intercept": [True, False]},
    "Lasso": {
        "alpha": [1, 0.1, 0.01, 0.001, 0.0001, 0],
        "fit_intercept": [True, False],
    },
    "ElasticNet": {
        "max_iter": [1, 5, 10],
        "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
        "l1_ratio": np.arange(0.0, 1.0, 0.1),
    },
    "RandomForestRegressor": {
        "bootstrap": [True, False],
        "max_depth": [2, 3, 5, 10, 20, 40, 50, 80, None],
        "max_features": ["auto", "sqrt"],
        "min_samples_leaf": [1, 2, 4],
        "min_samples_split": [2, 5, 10],
        "n_estimators": [
            10,
            50,
            100,
            200,
            400,
            800,
            1000,
            1400,
            1600,
            2000,
        ],
    },
    "KNeighborsRegressor": {
        "n_neighbors": [2, 4, 6, 8, 10, 16, 32],
        "weights": ["uniform", "distance"],
        "algorithm": [
            "auto",
            "ball_tree",
            "kd_tree",
            "brute",
        ],
    },
    "GradientBoostingRegressor": {
        "n_estimators": [20, 50, 100, 200],
        "learning_rate": [0.1, 0.05, 0.02, 0.01],
        "max_depth": [2, 3, 6, 10],
        "min_samples_leaf": [3, 6, 9, 15],
        "max_features": ["auto", "sqrt"],
    },
}


In [4]:
resp_df = pd.read_parquet("resp_df.parquet")

In [5]:
resp_df.head(10)

,app_1_containers,app_2_containers,app_3_containers,app_4_containers,CS_url_/2/2-2-2-3-2/197,CS_url_/2/2-2-2-4-4/72,CS_url_/2/2-2-3-2/126,CS_url_/2/2-2-4/199,CS_url_/2/2-2/6,CS_url_/2/2-2/95,CS_url_/2/2-3-2-2-3/104,CS_url_/2/2-3/104,CS_url_/2/2/119,CS_url_/2/2/130,CS_url_/2/2/247,CS_url_/2/2/57,CS_url_/2/3-2-3-4-2/94,CS_url_/2/3-2-3/93,CS_url_/2/3-2-4-4/40,CS_url_/2/3-3-2-3/126,CS_url_/2/3-3-2/245,CS_url_/2/3-3-3-3/125,CS_url_/2/3-3-3-3/84,CS_url_/2/3-3-3/181,CS_url_/2/3-3/87,CS_url_/2/3-3/91,CS_url_/2/3-4-2-3/152,CS_url_/2/3-4-3/155,CS_url_/2/3-4-4-4-3/154,CS_url_/2/3-4/105,CS_url_/2/3-4/232,CS_url_/2/3/188,CS_url_/2/3/6,CS_url_/2/4-2-2-3-2/204,CS_url_/2/4-2-4-2/65,CS_url_/2/4-2-4/235,CS_url_/2/4-3-2-2-2/105,CS_url_/2/4-3-2-2-2/85,CS_url_/2/4-3-4-2-4/12,CS_url_/2/4-3-4/76,CS_url_/2/4-3/65,CS_url_/2/4-4-2-3-4/103,CS_url_/2/4-4-2-3-4/222,CS_url_/2/4-4-2-4/104,CS_url_/2/4-4-3/147,CS_url_/2/4-4-4-2-3/49,CS_url_/2/4/23,CS_url_/3/2-2-3/49,CS_url_/3/2-3-2-2/204,CS_url_/3/2-3-2-2/212,CS_url_/3/2-3-2-3-3/93,CS_url_/3/2-3-3-2/107,CS_url_/3/2-3-4-4/11,CS_url_/3/2-3-4/201,CS_url_/3/2-4-3-3/13,CS_url_/3/2/158,CS_url_/3/3-2-2-2-2/197,CS_url_/3/3-2-2-2/137,CS_url_/3/3-2-2/77,CS_url_/3/3-2/82,CS_url_/3/3-3-2-2/115,CS_url_/3/3-3-3-3-3/101,CS_url_/3/3-3-3-4-4/14,CS_url_/3/3-3-3-4/9,CS_url_/3/3-3/110,CS_url_/3/3-4-4-4-4/215,CS_url_/3/3-4/115,CS_url_/3/3-4/230,CS_url_/3/3-4/37,CS_url_/3/3/139,CS_url_/3/3/14,CS_url_/3/3/180,CS_url_/3/3/47,CS_url_/3/4-2-2-4-4/26,CS_url_/3/4-2-2/38,CS_url_/3/4-2-2/96,CS_url_/3/4-2-3-4/56,CS_url_/3/4-2/223,CS_url_/3/4-3-3/93,CS_url_/3/4-3/42,CS_url_/3/4-4-2-4-4/215,CS_url_/3/4-4-3-4/66,CS_url_/3/4-4-3/163,CS_url_/3/4-4-3/60,CS_url_/3/4-4-4-4-2/37,CS_url_/3/4-4-4-4/217,CS_url_/3/4-4-4/139,CS_url_/3/4-4/192,CS_url_/3/4-4/195,CS_url_/3/4/3,CS_url_/3/4/71,CS_url_/4/2-2-4-4/145,CS_url_/4/2-3-3-4/64,CS_url_/4/2-3/65,CS_url_/4/2-4-2-2/75,CS_url_/4/2-4-3/155,CS_url_/4/2-4/147,CS_url_/4/2-4/79,CS_url_/4/2/110,CS_url_/4/2/228,CS_url_/4/2/229,CS_url_/4/2/88,CS_url_/4/3-2-4/33,CS_url_/4/3-3-3-3-3/181,CS_url_/4/3-3-3-3-4/16,CS_url_/4/3-4-3-4/122,CS_url_/4/3-4-3/96,CS_url_/4/3-4-4-3/216,CS_url_/4/3-4-4-4-3/183,CS_url_/4/3-4/154,CS_url_/4/3/169,CS_url_/4/3/171,CS_url_/4/3/188,CS_url_/4/3/65,CS_url_/4/4-2-4-3-2/45,CS_url_/4/4-3-2-4-4/139,CS_url_/4/4-3-3-4/229,CS_url_/4/4-3-3/238,CS_url_/4/4-3-4-4/149,CS_url_/4/4-3-4/142,CS_url_/4/4-3/224,CS_url_/4/4-3/76,CS_url_/4/4-4-2/41,CS_url_/4/4-4-3-4/16,CS_url_/4/4-4-4/102,CS_url_/4/4-4/214,CS_url_/4/4-4/69,CS_url_/4/4/106,CS_url_/4/4/130,CS_url_/4/4/223,CS_url_/4/4/231,CS_url_/4/4/71,CS_url_/4/4/79,av_rolling_resp_time_250
249,1.0,0.4,0.4,0.4,0.1,0.095238,0.111111,0.15,0.1875,0.000000,0.055556,0.1875,0.05,0.125,0.142857,0.117647,0.222222,0.105263,0.111111,0.0,0.222222,0.380952,0.055556,0.0625,0.058824,0.222222,0.0,0.0,0.166667,0.058824,0.0625,0.111111,0.058824,0.0,0.111111,0.105263,0.090909,0.058824,0.055556,0.0625,0.0625,0.0625,0.0,0.066667,0.263158,0.055556,0.347826,0.0,0.176471,0.0,0.038462,0.055556,0.0,0.0,0.000000,0.0,0.125,0.0625,0.117647,0.421053,0.125,0.2,0.1,0.260870,0.25,0.058824,0.277778,0.133333,0.0625,0.0,0.176471,0.611111,0.0625,0.125,0.380952,0.0,0.0625,0.058824,0.0,0.0,0.0625,0.0625,0.055556,0.058824,0.1875,0.058824,0.1250,0.0,0.0,0.105263,0.052632,0.176471,0.052632,0.0,0.176471,0.047619,0.133333,0.176471,0.235294,0.1875,0.125,0.0625,0.095238,0.15625,0.071429,0.0,0.058824,0.2,0.0,0.058824,0.117647,0.117647,0.238095,0.125,0.1250,0.0625,0.0,0.238095,0.0,0.0,0.117647,0.0,0.1,0.266667,0.117647,0.200000,0.0,0.133333,0.055556,0.176471,0.0,0.0,0.111111,0.009221
250,1.0,0.4,0.4,0.4,0.1,0.095238,0.111111,0.15,0.1875,0.000000,0.055556,0.1875,0.05,0.125,0.142857,0.117647,0.222222,0.105263,0.111111,0.0,0.222222,0.380952,0.055556,0.0625,0.058824,0.222222,0.0,0.0,0.166667,0.058824,0.0625,0.111111,0.058824,0.0,0.111111,0.105263,0.090909,0.058824,0.055556,0.0625,0.0625,0.0625,0.0,0.066667,0.263158,0.055556,0.347826,0.0,0.176471,0.0,0.038462,0.055556,0.0,0.0,0.000000,0.0,0.125,0.0625,0.11764

In [6]:
resp_df.shape

(4249883, 134)

In [7]:
y = resp_df.av_rolling_resp_time_250

In [8]:
X = resp_df.drop(columns=["av_rolling_resp_time_250"])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [10]:
X_train = np.ascontiguousarray(X_train)
y_train = np.ascontiguousarray(y_train)

In [11]:
X_test = np.ascontiguousarray(X_test)
y_test = np.ascontiguousarray(y_test)

In [ ]:
helper1 = EstimatorSelectionHelper(models1, params1)
helper1.fit(X_train, y_train, scoring='neg_mean_absolute_error', n_jobs=2)

Running GridSearchCV for Ridge.
Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
with open('super_results.pkl', 'wb') as f:
    pickle.dump(helper1, f)

with open('super_results_sum.pkl', 'wb') as f:
    pickle.dump(helper1.score_summary(sort_by='max_score'), f)
    

In [ ]:
helper1.score_summary(sort_by='max_score')